In [13]:
import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt

# relevant distributions
T = scipy.stats.t
Binomial = scipy.stats.binom

# get data
data = pd.read_csv("/Users/IKleisle/Downloads/runtimes.csv")
################################################
# Here's the missing step
################################################
data["runtime"] = np.log2(data["runtime"].values)
data.columns = [item.replace(".", "_") for item in data.columns]

# add on group-specific means
data = pd.merge(
    data,
    data.groupby(["alg_name", "prob_ind"], as_index=False)["runtime"].mean(),
    on=["alg_name", "prob_ind"],
    suffixes=["", "_bar"]
)

# total rows
N = data.shape[0]
M = len(data.alg_name.unique())
P = len(data.prob_ind.unique())
N_ = data.query("alg_name == 'A' & prob_ind == 1").shape[0]
# make sure dims all good
assert N_ * P * M == N
# precompute S2
S2 = (1 / (N - M * P)) * np.sum(
    (data["runtime"].values - data["runtime_bar"].values) ** 2
)

# just so I know what's in data
print(f"""Unique Algorithms: {data.alg_name.unique()}
Unique Problems: {data.prob_ind.unique()}""")

data.head()

Unique Algorithms: ['A' 'B' 'C' 'D' 'E']
Unique Problems: [1 2 3 4 5 6]


,alg_name,prob_ind,runtime,runtime_bar
0,A,1,1.223336,1.72526
1,A,1,0.485833,1.72526
2,A,1,1.856410,1.72526
3,A,1,1.589319,1.72526
4,A,1,1.002494,1.72526


### Part 1: T-Tests

In [14]:
def perform_t_test(model, ref_model="A"):
    """"""
    data_ref = data.query(f"alg_name == '{ref_model}'")
    data_mod = data.query(f"alg_name == '{model}'")

    y_bar_ref = data_ref.runtime.values.mean()
    y_bar_mod = data_mod.runtime.values.mean()

    assert N_ * P == data_ref.shape[0]
    assert N_ * P == data_mod.shape[0]

    t_stat = np.sqrt(N_ * P / 2) * (y_bar_ref - y_bar_mod) / np.sqrt(S2)
    pval = T(df=N - M * P).cdf(t_stat)

    return t_stat, pval

t_result = pd.DataFrame(
    [perform_t_test(i) for i in data.alg_name.unique()[1:]],
    columns=["t_statistic", "p_value"]
)
t_result["algorithm"] = data.alg_name.unique()[1:]

t_result

,t_statistic,p_value,algorithm
0,-13.834929,2.587548e-41,B
1,-8.566665,1.324226e-17,C
2,0.582763,7.199288e-01,D
3,-10.703225,4.260514e-26,E


In [15]:
# as a sanity check. 
data.groupby(["alg_name"], as_index=False)["runtime"].mean().rename(columns={"runtime": "mean_runtime"})

,alg_name,mean_runtime
0,A,-0.271135
1,B,0.438662
2,C,0.168376
3,D,-0.301033
4,E,0.277991


As given by the problem's composite Hypothesis, this is a one-sided (lower) test. As shown in the results above, we reject (for $\alpha = .05)$ the null hypothesis (that A has a 50/50 shot at outperforming its opponent) for the A vs. B, C, and E matchups -- this indicates that A may be comapratively better than these models. However, the model fails to reject for D, so we fail to reject the null hypothesis. 

And finally, the proclivity to reject here may reflect an overall inappropriate model -- perhaps our nulls and setup were way off base. 
### Part 2: Bernoulli-Test

In [16]:
def perform_general_test(model, ref_model="A"):
    """"""
    data_ref = data.query(f"alg_name == '{ref_model}'")
    data_mod = data.query(f"alg_name == '{model}'")
    bjk_results = (data_ref.runtime.values <= data_mod.runtime.values).astype(int)

    pval = 1 - Binomial(N_ * P, 1/2).cdf(np.sum(bjk_results))
    return np.sum(bjk_results), pval

In [17]:
gen_result = pd.DataFrame(
    [perform_general_test(i) for i in data.alg_name.unique()[1:]],
    columns=["Sum(Bjk)", "p_value"]
)
gen_result["opposing_algorithm"] = data.alg_name.unique()[1:]

gen_result

,Sum(Bjk),p_value,opposing_algorithm
0,252,1.110223e-16,B
1,231,1.110223e-16,C
2,169,1.209099e-02,D
3,244,1.110223e-16,E


Now, the hypothesis has been reversed, and the null is that the probability that Algorithm 1 is faster than Algorithm i is less than or equal to one-half -- this null is less believable, the more instances 1/0 in which 1 beats i occurs. As the first column in the table shows, algorithm A routinely beats the others, leading to a clean sweep of rejections at $\alpha = .05$. However, as this test does not explicitly correct for the $\beta$ variables (as the proper ANOVA setup does), this result may be misleading. 